## Residual Network

레지듀얼 네트워크(ResNets)를 사용하여 매우 깊은 컨볼루션 네트워크를 구축하는 방법을 배웁니다. 이론적으로 매우 깊은 네트워크는 매우 복잡한 기능을 나타낼 수 있습니다. 그러나 실제로는 훈련하기가 어렵습니다. 
[He et al.](https://arxiv.org/pdf/1512.03385.pdf)에서 소개한 Residual Networks를 사용하면 이전에 실제로 가능했던 것보다 훨씬 더 깊은 네트워크를 훈련시킬 수 있습니다.

**이 과제에서 다음을 수행합니다.**

- ResNets의 기본 구성 요소를 구현합니다.
- 이러한 구성 요소를 결합하여 이미지 분류를 위한 최첨단 신경망을 구현하고 훈련합니다.

이 과제는 tensorflow.keras 에서 수행됩니다.

문제로 뛰어들기 전에 아래 셀을 실행하여 필요한 패키지를 로드해 보겠습니다.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
#### Necessary Imports for Neural Net 

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D,\
     Flatten, BatchNormalization, AveragePooling2D, Dense, Activation, Add 
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from resnets_utils import *

import tensorflow.keras.backend as K

### 1-심층 신경망의 문제

지난 주에 첫 번째 컨볼루션 신경망을 구축했습니다. 최근 몇 년 동안 신경망은 더 깊어졌으며 최첨단 네트워크가 단 몇 개의 층(예: AlexNet)에서 100 개 이상의 층으로 이동했습니다.

매우 심층적인 네트워크의 주요 이점은 매우 복잡한 기능을 나타낼 수 있다는 것입니다. 또한 에지(하위 계층)에서 매우 복잡한 기능(더 깊은 계층)에 이르기까지 다양한 추상화 수준에서 기능을 학습 할 수 있습니다. 그러나 더 깊은 네트워크를 사용하는 것이 항상 도움이 되는 것은 아닙니다. 이를 훈련하는 데 큰 장벽은 그라디언트가 사라지는 것입니다. 매우 깊은 네트워크는 종종 그라디언트 신호가 빠르게 0으로 이동하여 그라디언트 하강을 견딜 수 없을 정도로 느리게 만듭니다. 더 구체적으로 말하면, 경사하강법 동안 최종 층에서 첫 번째 층으로 역전파 할 때 각 단계에서 가중치 행렬을 곱하면 경사가 기하 급수적으로 빠르게 0으로 감소 할 수 있습니다 (또는 드물게 기하 급수적으로 증가 할 수 있음). 신속하게 "폭발"하여 매우 큰 값을 취합니다.)

따라서 훈련 중에 이전 계층에 대한 기울기의 크기(또는 표준)가 훈련이 진행됨에 따라 매우 빠르게 0으로 감소하는 것을 볼 수 있습니다.

<img src="images/vanishing_grad_kiank.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='purple'> Figure 1 </u><font color='purple'>  : 소멸 그라디언트 <br> 네트워크 학습에 따라 초기 계층의 학습 속도가 매우 빠르게 감소합니다. </center></caption>

이제 잔여 네트워크를 구축하여 이 문제를 해결할 것입니다!

### 2-잔여 네트워크 구축

ResNets에서 "바로 가기" 또는 "건너뛰기 연결"을 사용하면 그래디언트를 이전 레이어로 직접 역전파 할 수 있습니다.

<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='purple'> Figure 2 </u><font color='purple'>  : 건너뛰기 연결(skip-connection)을 보여주는 ResNet 블록 <br> </center></caption>

왼쪽 이미지는 네트워크를 통한 "주 경로"를 보여줍니다. 오른쪽 이미지는 기본 경로에 바로 가기를 추가합니다. 이러한 ResNet 블록을 서로 쌓아두면 매우 깊은 네트워크를 형성 할 수 있습니다.

또한 강의에서 바로 가기가 있는 ResNet 블록을 사용하면 블록 중 하나가 식별 기능을 매우 쉽게 학습 할 수 있음을 확인했습니다. 즉, 훈련 세트 성능에 해를 끼칠 위험이 거의 없이 추가 ResNet 블록에 쌓을 수 있습니다. (그래디언트가 사라지는 데 도움이 되는 연결을 건너 뛰는 것보다 훨씬 더 쉽게 식별 기능을 배우는 것이 ResNets의 놀라운 성능을 설명한다는 증거도 있습니다.)

ResNet에서는 주로 입력/출력 치수가 동일한 지 다른 지에 따라 두 가지 주요 유형의 블록이 사용됩니다. 둘 다 구현할 것입니다.

#### 2.1- 단위(Identity) 블록

단위 블록은 ResNets에서 사용되는 표준 블록이며 입력 활성화(예: $a^{[l]}$)가 출력 활성화와 동일한 차원(예: $a^{[l + 2]}$)을 갖는 경우에 해당합니다. ResNet의 단위 블록에서 발생하는 여러 단계를 구체화하기 위해 다음은 개별 단계를 보여주는 대체 다이어그램입니다.

<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> Figure 3 </u><font color='purple'>  : 단위 블록(Identity block). 건너뛰기 연결(skip-connection)은 2층을 "건너 뜁니다". </center></caption>

위쪽 경로는 "바로 가기 경로"입니다. 아래쪽 경로는 "주 경로"입니다. 이 다이어그램에서는 각 계층의 CONV2D 및 ReLU 단계도 명시했습니다. 훈련 속도를 높이기 위해 BatchNorm 단계도 추가했습니다. 이것이 구현하기 복잡하다는 것에 대해 걱정하지 마십시오. BatchNorm은 Keras의 코드 한 줄에 불과하다는 것을 알 수 있습니다!

이 연습에서는 스킵 연결이 2 개의 레이어가 아닌 3 개의 히든 레이어를 "건너뛰는" 이 단위 블록의 약간 더 강력한 버전을 실제로 구현합니다. 다음과 같이 보입니다.

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 4** </u><font color='purple'>  : 단위 블록(Identity block). 건너뛰기 연결(skip-connection)은 3층을 "건너 뜁니다".</center></caption>

다음은 개별 단계입니다.

기본 경로의 첫 번째 구성 요소 :

- 첫 번째 CONV2D에는 모양 (1,1) 및 스트라이드 (1,1)의 $F_1$ 필터가 있습니다. 패딩은 "유효"하고 이름은 `conv_name_base + '2a'` 여야 합니다. 무작위 초기화를 위한 시드로 0을 사용하십시오.
- 첫 번째 BatchNorm은 채널 축을 정규화합니다. 이름은 `bn_name_base + '2a'` 여야합니다.
- ReLU 활성화 기능을 적용합니다. 이름과 하이퍼 파라미터가 없습니다.

기본 경로의 두 번째 구성 요소 :

- 두 번째 CONV2D에는 $(f,f)$ 모양의 $F_2$ 필터와 (1,1) 보폭이 있습니다. 패딩은 "same"이고 이름은 `conv_name_base + '2b'` 여야 합니다. 무작위 초기화를 위한 시드로 0을 사용하십시오.
- 두 번째 BatchNorm은 채널 축을 정규화합니다. 이름은 `bn_name_base + '2b'` 여야합니다.
- ReLU 활성화 기능을 적용합니다. 이름과 하이퍼 파라미터가 없습니다.

주 경로의 세 번째 구성 요소 :

- 세 번째 CONV2D에는 $F_3$ 모양의 필터 (1,1)와 보폭 (1,1)이 있습니다. 패딩은 "유효"하고 이름은 `conv_name_base + '2c'` 여야 합니다. 무작위 초기화를 위한 시드로 0을 사용하십시오.
- 세 번째 BatchNorm은 채널 축을 정규화합니다. 이름은 `bn_name_base + '2c'` 여야합니다. 이 구성 요소에는 ReLU 활성화 기능이 없습니다.

마지막 단계:

- 바로 가기와 입력이 함께 추가됩니다.
- ReLU 활성화 기능을 적용합니다. 이름과 하이퍼 파라미터가 없습니다.

**(1) 연습문제** : ResNet ID 블록을 구현합니다. 우리는 주 경로의 첫 번째 구성 요소를 구현했습니다. 이것이 무엇을 하고 있는지 이해하기 위해이 내용을 주의 깊게 읽으십시오. 나머지는 구현해야 합니다.

- Conv2D 단계 구현 : [참고 자료보기](https://keras.io/layers/convolutional/#conv2d)
- BatchNorm 구현 방법 : [참고 자료보기](https://faroit.github.io/keras-docs/1.2.2/layers/normalization/) (축 : Integer, 정규화해야하는 축 (일반적으로 채널 축) )
- 활성화를 위해 `Activation('relu')(X)`를 사용합니다.
- 바로 가기로 전달 된 값을 추가하려면 : [참고 자료보기](https://keras.io/layers/merge/#add)

In [2]:
# identity block

def res_identity(x, filters): 
    '''
    renet block where dimension does not change.
    The skip connection is just simple identity conncection
    we will have 3 blocks and then input will be added
    '''
    
    x_skip = x # this will be used for addition with the residual block 
    f1, f2, f3 = filters

    #first block 
    x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)

    
    

    # Third component of main path (≈2 lines)

    
    

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)

    

    ### END CODE HERE ###

    return x

In [3]:
# testing the identity block 
tf.random.set_seed(1)
input_shape = (3, 4, 4, 6)
x = tf.random.normal(input_shape)
A = res_identity(x, filters=[2,4,6])
print('A.shape : ', A.shape, '\nA[:][1][1][0] =', A[:][1][1][1])

A.shape :  (3, 4, 4, 6) 
A[:][1][1][0] = tf.Tensor([0.491265   0.         0.45889473 0.11222664 0.         0.45041496], shape=(6,), dtype=float32)


### 2.2 - 컨볼루션 블록

ResNet 단위 블록을 구현했습니다. 다음으로 ResNet "convolutional block"은 다른 유형의 블록입니다. 입력 및 출력 차원이 일치하지 않을 때 이 유형의 블록을 사용할 수 있습니다. 단위 블록과의 차이점은 바로 가기 경로에 CONV2D 레이어가 있다는 것입니다.

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 4** </u><font color='purple'>  : 컨볼루션 블록 </center></caption>


바로 가기 경로의 CONV2D 레이어는 입력 $x$의 크기를 다른 차원으로 조정하는 데 사용되므로 기본 경로에 바로 가기 값을 다시 추가하는 데 필요한 최종 추가에서 차원이 일치합니다. (이것은 강의에서 논의한 $W_s$ 행렬과 비슷한 역할을 합니다.) 예를 들어 활성화 차원의 높이와 너비를 2 배로 줄이려면 보폭이 2 인 1x1 컨볼루션을 사용할 수 있습니다. CONV2D 레이어 바로 가기 경로는 비선형 활성화 기능을 사용하지 않습니다. 주요 역할은 입력의 차원을 줄이는(학습된) 선형 함수를 적용하여 차원이 이후의 추가 단계와 일치하도록 하는 것입니다.

컨볼루션 블록의 세부 사항은 다음과 같습니다.

기본 경로의 첫 번째 구성 요소 :

- 첫 번째 CONV2D에는 $F_1$ 모양의 필터 (1,1) 및 스트라이드 (s, s)가 있습니다. 패딩은 "유효"하고 이름은 `conv_name_base + '2a'` 여야 합니다.
- 첫 번째 BatchNorm은 채널 축을 정규화합니다. 이름은 `bn_name_base + '2a'` 여야합니다.
- ReLU 활성화 기능을 적용합니다. 이름과 하이퍼 파라미터가 없습니다.

기본 경로의 두 번째 구성 요소 :
- 두 번째 CONV2D에는 (f, f)의 $F_2$ 필터와 (1,1)의 스트라이드가 있습니다. 패딩은 "same"이고 이름은 `conv_name_base + '2b'` 여야 합니다.
- 두 번째 BatchNorm은 채널 축을 정규화합니다. 이름은 `bn_name_base + '2b'` 여야합니다.
- ReLU 활성화 기능을 적용합니다. 이름과 하이퍼 파라미터가 없습니다.

주 경로의 세 번째 구성 요소 :
- 세 번째 CONV2D에는 (1,1)의 $F_3$ 필터와 (1,1)의 스트라이드가 있습니다. 패딩은 "유효"하며 이름은 `conv_name_base + '2c'` 여야 합니다.
- 세 번째 BatchNorm은 채널 축을 정규화합니다. 이름은 `bn_name_base + '2c'` 여야합니다. 이 구성 요소에는 ReLU 활성화 기능이 없습니다.

바로 가기 경로 :
- CONV2D에는 $F_3$ 모양의 필터 (1,1) 및 스트라이드 (s, s)가 있습니다. 패딩은 "유효"하고 이름은 `conv_name_base + '1'`이어야 합니다.
- BatchNorm이 채널 축을 정규화하고 있습니다. 이름은 `bn_name_base + '1'`이어야 합니다.

마지막 단계:
- 바로 가기와 주요 경로 값이 함께 추가됩니다.
- ReLU 활성화 기능을 적용합니다. 이름과 하이퍼 파라미터가 없습니다.
    
**(2) 연습문제** : 컨볼루션 블록을 구현합니다. 우리는 주 경로의 첫 번째 구성 요소를 구현했습니다. 나머지는 구현해야 합니다. 이전과 마찬가지로 그레이더와의 일관성을 보장하기 위해 항상 0을 무작위 초기화의 시드로 사용합니다.
- [Conv Hint](https://keras.io/layers/convolutional/#conv2d)
- [BatchNorm Hint](https://keras.io/layers/normalization/#batchnormalization) (axis : Integer, 정규화해야하는 축 (일반적으로 기능 축))
- 활성화를 위해`Activation ( 'relu') (X)`를 사용합니다.
- [추가 힌트](https://keras.io/layers/merge/#add)

In [4]:
# convolutional block

def res_conv(x, s, filters):
    '''
    here the input size changes
    ''' 
    x_skip = x
    f1, f2, f3 = filters

    # first block
    x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
    # when s = 2 then it is like downsizing the feature map
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)

    
    

    # Third component of main path (≈2 lines)

    
    

    ##### SHORTCUT PATH #### (≈2 lines)

    
    

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)

    
    

    ### END CODE HERE ###
    return x

In [5]:
# testing the convolution block 
tf.random.set_seed(1)
input_shape = (3, 4, 4, 6)
x = tf.random.normal(input_shape)
A = res_conv(x, s=2, filters=[2,4,6])
print('A.shape : ', A.shape, '\nA[:][0][0][0] =', A[:][0][0][0])

A.shape :  (3, 2, 2, 6) 
A[:][0][0][0] = tf.Tensor([0.         1.5257459  0.         0.43303442 1.5155082  0.        ], shape=(6,), dtype=float32)


### 3- 첫 번째 ResNet 모델 구축 (50 개 레이어)

이제 매우 깊은 ResNet을 구축하는 데 필요한 블록이 있습니다. 다음 그림은 이 신경망의 아키텍처를 자세히 설명합니다. 다이어그램에서 "ID BLOCK"은 "Identity block"을 의미하고 "ID BLOCK x3"은 3 개의 단위 블록을 함께 쌓아야 함을 의미합니다.

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'> Figure 5 </u><font color='purple'>  : ResNet-50 model </center></caption>

이 ResNet-50 모델의 세부 사항은 다음과 같습니다.

- 제로 패딩은 입력을 (3,3) 패드로 채웁니다.
- 스테이지 1:
    - 2D 컨볼루션에는 64 개의 모양 필터 (7,7)가 있으며 스트라이드 (2,2)를 사용합니다. 이름은 "conv1"입니다.
    - BatchNorm은 입력 채널 축에 적용됩니다.
    - MaxPooling은 (3,3) 창과 (2,2) 보폭을 사용합니다.
- 2 단계 :
    - 컨볼루션 블록은 [64,64,256], "f"는 3, "s"는 1, 블록은 "a"의 세 가지 필터 세트를 사용합니다.
    - 2 개의 단위 블록은 [64,64,256], "f"는 3, 블록은 "b"및 "c"의 세 가지 필터 세트를 사용합니다.
- 3 단계 :
    - 컨볼루션 블록은 [128,128,512], "f"는 3, "s"는 2, 블록은 "a"의 세 가지 필터 세트를 사용합니다.
    - 3 개의 단위 블록은 [128,128,512], "f"는 3이고 블록은 "b", "c"및 "d"의 세 가지 필터 세트를 사용합니다.
- 4 단계 :
    - 컨볼루션 블록은 [256, 256, 1024], "f"는 3, "s"는 2, 블록은 "a"의 세 가지 필터 세트를 사용합니다.
    - 5 개의 단위 블록은 크기 [256, 256, 1024]의 세 가지 필터 세트를 사용하고 "f"는 3이고 블록은 "b", "c", "d", "e"및 "f"입니다.
- 5 단계 :
    - 컨볼루션 블록은 [512, 512, 2048], "f"는 3, "s"는 2, 블록은 "a"의 세 가지 필터 세트를 사용합니다.
    - 2 개의 단위 블록은 [512, 512, 2048], "f"는 3이고 블록은 "b"와 "c"의 세 가지 필터 세트를 사용합니다.
- 2D 평균 풀링은 모양 (2,2)의 창을 사용하며 이름은 "avg_pool"입니다.
- 평면화에는 하이퍼 파라미터나 이름이 없습니다.
- Fully Connected (Dense) 레이어는 소프트 맥스 활성화를 사용하여 입력을 클래스 수로 줄입니다. 이름은` 'fc'+ str (classes)`이어야 합니다.

**(3) 연습문제** : 위 그림에 설명된 50 개의 레이어로 ResNet을 구현합니다. 1 단계와 2 단계를 구현했습니다. 나머지는 구현하십시오. (단계 3-5를 구현하는 구문은 단계 2의 구문과 매우 유사해야합니다.) 위 텍스트의 명명 규칙을 따라야합니다.

이 기능을 사용해야 합니다.
- 평균 풀링 [참고 자료 참조](https://keras.io/layers/pooling/#averagepooling2d)

아래 코드에서 사용한 다른 함수는 다음과 같습니다.
- Conv2D : [참고 자료보기](https://keras.io/layers/convolutional/#conv2d)
- BatchNorm : [참조 참조](https://keras.io/layers/normalization/#batchnormalization) (axis : Integer, 정규화해야하는 축 (일반적으로 기능 축))
- 제로 패딩 : [참고 자료보기](https://keras.io/layers/convolutional/#zeropadding2d)
- Max pooling : [참고 자료보기](https://keras.io/layers/pooling/#maxpooling2d)
- 완전 연결 레이어 : [참고 자료 참조](https://keras.io/layers/core/#dense)
- 추가 : [참고 자료보기](https://keras.io/layers/merge/#add)

In [6]:
# ResNet50

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    input_im = Input(input_shape) # cifar 10 images size
    x = ZeroPadding2D(padding=(3, 3))(input_im)

    # 1st stage
    # here we perform maxpooling, see the figure above

    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    #2nd stage 
    # frm here on only conv block and identity block, no pooling

    x = res_conv(x, s=1, filters=(64, 64, 256))
    x = res_identity(x, filters=(64, 64, 256))
    x = res_identity(x, filters=(64, 64, 256))

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    x = res_conv(x, s=2, filters=(128, 128, 512))
    x =  
    x = 
    x =  

    # Stage 4 (≈6 lines)
    x = res_conv(x, s=2, filters=(256, 256, 1024))
    x =  
    x = 
    x = 
    x =  
    x =  

    # Stage 5 (≈3 lines)
    x = res_conv(x, s=2, filters=(512, 512, 2048))
    x =  
    x =  
    
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"

    x =  
    
    ### END CODE HERE ###

    x = Flatten()(x)
    x = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

    # define the model 

    model = Model(inputs=input_im, outputs=x, name='Resnet50')

    return model

다음 코드를 실행하여 모델의 그래프를 작성하십시오. 구현이 올바르지 않으면 아래`model.fit (...)`을 실행할 때 정확성을 확인하여 알 수 있습니다.

In [7]:
model = ResNet50()

Keras Tutorial Notebook에서 볼 수 있듯이 모델을 학습하기 전에 모델을 컴파일하여 학습 프로세스를 구성해야 합니다.

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

이제 모델을 학습 SIGNS 데이터 셋을로드 해 보겠습니다.할 준비가 되었습니다. 필요한 것은 데이터 세트뿐입니다.

SIGNS 데이터셋을 로드 해 보겠습니다.
<img src="images/signs_data_kiank.png" style="width:450px;height:250px;">
<caption><center> <u> <font color='purple'> Figure 6 </u><font color='purple'>  : SIGNS dataset </center></caption>

In [9]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


다음 셀을 실행하여 배치 크기가 32 인 5 Epoch에서 모델을 훈련합니다. CPU에서는 Epoch 당 약 5 분 정도 걸립니다.

In [10]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 32)

Train on 1080 samples
Epoch 1/5
1080/1080 [==============================] - 180s 167ms/sample - loss: 19.8641 - accuracy: 0.5111
Epoch 2/5
1080/1080 [==============================] - 154s 143ms/sample - loss: 18.8262 - accuracy: 0.7778
Epoch 3/5
1080/1080 [==============================] - 155s 144ms/sample - loss: 18.0315 - accuracy: 0.8685
Epoch 4/5
1080/1080 [==============================] - 320s 296ms/sample - loss: 17.3114 - accuracy: 0.8843
Epoch 5/5
1080/1080 [==============================] - 159s 148ms/sample - loss: 16.3313 - accuracy: 0.9426


**Expected Output**:

<table>
    <tr>
        <td>
            Epoch 5/5
        </td>
        <td>
           loss: between 16 and 17, acc: between 0.8 and 0.95, you should see your loss decreasing and the accuracy increasing.
        </td>
    </tr>

</table>

이 모델(에포크 5개만 훈련 됨)이 테스트 세트에서 어떻게 수행되는지 살펴 보겠습니다.

In [11]:
preds = model.evaluate(X_test, Y_test, verbose = 0)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

Loss = 19.447076670328777
Test Accuracy = 0.15


**Expected Output**:

<table>
    <tr>
        <td>
            Test Accuracy
        </td>
        <td>
           between 0.1 and 0.2
        </td>
    </tr>
</table>

이 과제의 목적을 위해 5 시대에 대해서만 모델을 훈련하도록 요청했습니다. 성능이 좋지 않음을 알 수 있습니다. 
이 과제의 공식 (채점) 부분을 완료 한 후 원하는 경우 추가 반복을 위해 ResNet을 선택적으로 훈련 할 수도 있습니다. 우리는 ~ 20 epoch 동안 훈련하면 훨씬 더 나은 성능을 얻지 만 CPU에서 훈련 할 때는 1 시간 이상이 걸립니다.

ResNet50은 적절한 반복 횟수에 대해 훈련 된 경우 이미지 분류를 위한 강력한 모델입니다. 학습한 내용을 사용하고 이를 자신의 분류 문제에 적용하여 최첨단 정확도를 수행 할 수 있기를 바랍니다.

이 과제를 마치신 것을 축하드립니다! 이제 최첨단 이미지 분류 시스템을 구현했습니다!

### 4-자신의 이미지로 테스트 (선택 사항 / 미 채점)

원하는 경우 자신의 손 사진을 찍고 모델 출력을 볼 수도 있습니다. 이것을 하기 위해:
     1. 이 노트북의 상단 표시 줄에서 "파일"을 클릭 한 다음 "열기"를 클릭하여 Coursera Hub로 이동합니다.
     2. "images"폴더에 있는 이 Jupyter Notebook의 디렉토리에 이미지를 추가합니다.
     3. 다음 코드에 이미지 이름을 작성합니다.
     4. 코드를 실행하고 알고리즘이 올바른지 확인하십시오!

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import scipy.misc
from matplotlib.pyplot import imshow

img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)
my_image = scipy.misc.imread(img_path)
imshow(my_image)
print("class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ")
print(model.predict(x))

다음 코드를 실행하여 모델 요약을 인쇄 할 수도 있습니다.

In [ ]:
model.summary()

마지막으로 아래 코드를 실행하여 ResNet50을 시각화합니다. "파일-> 열기 ...-> model.png"로 이동하여 모델의 .png 사진을 다운로드 할 수도 있습니다.

In [ ]:
from IPython.display import SVG
from tenkeras.utils import plot_model

plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

**기억해야 할 사항:**
- 매우 깊은 "일반" 네트워크는 그라디언트가 사라지기 때문에 훈련하기가 어렵기 때문에 실제로 작동하지 않습니다.
- 건너뛰기 연결은 소실 그라데이션 문제를 해결하는 데 도움이 됩니다. 또한 ResNet 블록이 단위 기능을 쉽게 배울 수 있습니다.
- 블록에는 단위 블록과 컨볼루션 블록의 두 가지 주요 유형이 있습니다.
- 이러한 블록을 함께 쌓아 매우 깊은 잔여 네트워크를 구축합니다.

### 참조

이 노트북은 He 등으로 인해 ResNet 알고리즘을 제시합니다. (2015). 여기의 구현은 또한 상당한 영감을 얻었으며 Francois Chollet의 github 저장소에 제공된 구조를 따릅니다.

- Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun-[Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)
- Francois Chollet의 github 저장소 : https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py
- ResNet in tensorflow 2.0 github 저장소 : https://github.com/suvoooo/Learn-TensorFlow/blob/master/resnet/Implement_Resnet_TensorFlow.ipynb 